In [7]:
import psycopg2
from datetime import datetime

# PostgreSQL database credentials
DB_CONFIG = {
    "dbname": "rag_db",
    "user": "user",
    "password": "password",
    "host": "192.168.0.34",
    "port": 5432
}

def fetch_highest_similarity_with_last_quantum():
    query = """
    WITH quantum_numbers AS (
        SELECT 
            id AS quantum_id,
            generated_at AS quantum_generated_at,
            process_id
        FROM 
            powerball_ticket_sessions
        WHERE 
            source = 'quantum'
            AND DATE(generated_at) IN ('2024-12-19', '2024-12-20')
        ORDER BY 
            process_id, generated_at
    ),
    ticket_similarity AS (
        SELECT 
            pts.id AS ticket_id,
            pts.generated_at AS ticket_generated_at,
            pts.embedding <-> ARRAY[
                wn.white_ball_1 / 69.0,
                wn.white_ball_2 / 69.0,
                wn.white_ball_3 / 69.0,
                wn.white_ball_4 / 69.0,
                wn.white_ball_5 / 69.0,
                wn.powerball / 26.0
            ]::vector AS similarity,
            pts.process_id
        FROM 
            powerball_ticket_sessions pts
        CROSS JOIN 
            winning_numbers wn
        WHERE 
            DATE(pts.generated_at) IN ('2024-12-19', '2024-12-20')
    ),
    last_previous_quantum AS (
        SELECT 
            q1.quantum_id AS current_quantum_id,
            q1.quantum_generated_at AS current_quantum_time,
            q2.quantum_id AS previous_quantum_id,
            q2.quantum_generated_at AS previous_quantum_time,
            q1.process_id,
            ROW_NUMBER() OVER (PARTITION BY q1.quantum_id ORDER BY q2.quantum_generated_at DESC) AS rn
        FROM 
            quantum_numbers q1
        LEFT JOIN 
            quantum_numbers q2
        ON 
            q1.process_id = q2.process_id
            AND q2.quantum_generated_at < q1.quantum_generated_at
        WHERE 
            q2.quantum_id IS NOT NULL
    )
    SELECT 
        lpq.current_quantum_id,
        lpq.current_quantum_time,
        lpq.previous_quantum_id,
        lpq.previous_quantum_time,
        ts.ticket_id,
        ts.similarity,
        ts.process_id
    FROM 
        last_previous_quantum lpq
    LEFT JOIN 
        ticket_similarity ts
    ON 
        ts.process_id = lpq.process_id
        AND ts.ticket_generated_at BETWEEN lpq.previous_quantum_time AND lpq.current_quantum_time
    WHERE 
        ts.similarity IS NOT NULL
        AND lpq.rn = 1
    ORDER BY 
        ts.similarity ASC
    LIMIT 20;
    """
    try:
        # Connect to the database
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)
        results = cursor.fetchall()

        # Close the connection
        cursor.close()
        conn.close()

        return results
    except Exception as e:
        print(f"Database error: {e}")
        return []

def main():
    results = fetch_highest_similarity_with_last_quantum()
    if results:
        print("Top 20 Results with Immediate Previous Quantum Number:")
        for row in results:
            current_quantum_id, current_quantum_time, previous_quantum_id, previous_quantum_time, ticket_id, similarity, process_id = row
            print(f"Current Quantum ID: {current_quantum_id}, Previous Quantum ID: {previous_quantum_id}, "
                  f"Ticket ID: {ticket_id}, Similarity: {similarity:.5f}, Process ID: {process_id}, "
                  f"Time Interval: {current_quantum_time - previous_quantum_time}")
    else:
        print("No results found.")

if __name__ == "__main__":
    main()


Top 20 Results with Immediate Previous Quantum Number:
Current Quantum ID: 755, Previous Quantum ID: 245, Ticket ID: 358, Similarity: 0.03834, Process ID: 337816, Time Interval: 1:12:48.481125
Current Quantum ID: 915, Previous Quantum ID: 770, Ticket ID: 911, Similarity: 0.05431, Process ID: 337816, Time Interval: 0:20:11.230418
Current Quantum ID: 755, Previous Quantum ID: 245, Ticket ID: 601, Similarity: 0.06149, Process ID: 337816, Time Interval: 1:12:48.481125
Current Quantum ID: 2233, Previous Quantum ID: 2152, Ticket ID: 2173, Similarity: 0.07100, Process ID: 460803, Time Interval: 0:12:06.915724
Current Quantum ID: 1238, Previous Quantum ID: 1080, Ticket ID: 1085, Similarity: 0.07253, Process ID: 337816, Time Interval: 0:22:14.761205
Current Quantum ID: 755, Previous Quantum ID: 245, Ticket ID: 561, Similarity: 0.08069, Process ID: 337816, Time Interval: 1:12:48.481125
Current Quantum ID: 2109, Previous Quantum ID: 1881, Ticket ID: 2017, Similarity: 0.08204, Process ID: 460803, 